In [8]:
import requests
import json
import pandas as pd
import os

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pprint

from dataExtract import DataExtraction
obj_extraction = DataExtraction()


In [9]:
"""< InteractiveDebuggingTool. >
Get and display information events from the corresponding json file according to the widget's options 
:w_gameType:    dropdown widget,    fixed_options = ['regular', 'play-off']
:w_year:        dropdown widget,    fixed_options = ['2016', '2017', '2018', '2019', '2020']
:w_game_id:     IntSlider widget,   fixed_min = 1 ; max = len(game_ids_list) in <year> and <gameType>
:w_events:      IntSlider widget,   fixed_min = 1 ; max = len(events_ids_list) in <gameId> <year> and <gameType>
:rtype: <widgets, plot, text information> 
:return: Interactive widgets: Display key information such as puck coordinates for each event.
"""
#dropdown widget 
w_gameType = widgets.Dropdown(
    options=['regular', 'play-off'],
    description='game_type',
    disabled=False,
)
#dropdown widget 
w_year = widgets.Dropdown(
    options=['2016', '2017', '2018', '2019', '2020'],
    description='year',
    disabled=False,
)
#IntSlider widget game_id, and value <game_id_max> will be influence by <gameType> and <year> 
w_game_id = widgets.IntSlider(
    min= 1,
    max = 1271,
    description='game_number',
    disabled=False,
    style={'description_width': 'initial'}
)
#IntSlider widget events, and value <events_max> will be influence by <gameType>, <year> and <game_id> 
w_events = widgets.IntSlider(
    min= 1,
    max = 100,
    description='events',
    disabled=False,
)
u1 = widgets.HBox([w_gameType, w_year])
u2 = widgets.HBox([w_game_id, w_events])
ui = widgets.VBox([u1, u2])
#display(w_gameType, w_year)


def f(w_year, w_gameType, w_game_id, w_events):
    '''
    Display data and puck position
    game_data -> .json
    get all information from game_data
    '''
    game_type = 'R' if w_gameType == 'regular' else 'P'
    game_ids_list = obj_extraction.get_game_ids(game_type, w_year + str(int(w_year)+1))
    game_data = obj_extraction.fetch_data(w_year, game_type, game_ids_list[w_game_id - 1][-4:])

    #team_name_abbreviation
    team_home = game_data["gameData"]["teams"]["home"]["abbreviation"]
    team_away = game_data["gameData"]["teams"]["away"]["abbreviation"]
    #goal
    team_home_goals = game_data['liveData']["linescore"]["teams"]["home"]["goals"]
    team_away_goals = game_data['liveData']["linescore"]["teams"]["away"]["goals"]
    #shotsOnGoal
    team_home_sog = game_data['liveData']["linescore"]["teams"]["home"]["shotsOnGoal"]
    team_away_sog = game_data['liveData']["linescore"]["teams"]["away"]["shotsOnGoal"]
    #SO Goals
    team_home_SO_goal=game_data['liveData']["linescore"]["shootoutInfo"]["home"]["scores"]
    team_away_SO_goal=game_data['liveData']["linescore"]["shootoutInfo"]["away"]["scores"]
    #SO Attempts
    team_home_SO_att=game_data['liveData']["linescore"]["shootoutInfo"]["home"]["attempts"]
    team_away_SO_att=game_data['liveData']["linescore"]["shootoutInfo"]["away"]["attempts"]
    #event_time
    event_time = game_data['liveData']['plays']['allPlays'][w_events - 1]["about"]["periodTime"]
    event = game_data['liveData']['plays']['allPlays'][w_events - 1]["result"]["description"]
    #period_info [1, 2, 3]
    period_time = game_data['liveData']['plays']['allPlays'][w_events - 1]["about"]["periodTime"]
    period = game_data['liveData']['plays']['allPlays'][w_events - 1]["about"]["period"]

    #str game_info##################################################################################
    print('game_id \t'+ game_ids_list[w_game_id-1])
    print('total games \t'+ str(len(game_ids_list)))
    print('total events \t' + str(len(game_data['liveData']['plays']['allPlays'])))
    print('dateTime \t'+game_data["gameData"]["datetime"]["dateTime"])

    #table game_info################################################################################
    data = {'Home': [
        team_home,
        team_home_goals,
        team_home_sog,
        team_home_SO_goal,
        team_home_SO_att
        ],
     'Away': [
        team_away,
        team_away_goals,
        team_away_sog,
        team_away_SO_goal,
        team_away_SO_att
        ]
    }
    df = pd.DataFrame(data, index= ['Teams:','Goals:','SoG:','SO Goals:','SO Attempts'])
    print(df)

    #plot_information##############################################################################
    fig = plt.figure(figsize=(8, 4))
    img = matplotlib.image.imread('./images/input/nhl_rink.png')
    fig, ax = plt.subplots()
    ax.imshow(img,extent=[-100, 100, -42.5, 42.5])

    plt.xlim((-100, 100))
    plt.ylim((-42.5, 42.5))
    my_y_ticks = np.arange(-42.5, 42.5, 21.25)
    plt.yticks(my_y_ticks)
    plt.xlabel('feet')
    plt.ylabel('feet')

    
    if period % 2 == 0:#switch sides when P = 2 4 6 8
        plt.title(event + '\n' + period_time + '  P-' + str(period) + '\n'+ team_home + ' '*40 + team_away)
    else:
        plt.title(event + '\n' + period_time + '  P-' + str(period) + '\n'+ team_away + ' '*40 + team_home)

    try:
        X = game_data['liveData']['plays']['allPlays'][w_events - 1]['coordinates']['x']
        Y = game_data['liveData']['plays']['allPlays'][w_events - 1]['coordinates']['y']
        #print(X, Y)
        plt.plot([X],[Y],'bo')
        plt.annotate(' <'+str(X)+', '+str(Y)+'>', xy=(X, Y))

    except:
        plt.text(-50, 0, "! no puck on the field at this time !")
        #print("There is no puck on the field at this time !")
    
    plt.show()
    #print play_info##############################################################################
    play_info = game_data['liveData']['plays']['allPlays'][w_events - 1]
    pprint.pprint(play_info)


    

#question: Is it possible to reuse variables in every func???
#update game_id_max
def update_w_game(*arg):
    game_type = 'R' if w_gameType.value == 'regular' else 'P'
    game_ids_list = obj_extraction.get_game_ids(game_type, w_year.value + str(int(w_year.value)+1))

    w_game_id.max = len(game_ids_list)
w_gameType.observe(update_w_game, names='value')
w_year.observe(update_w_game, names='value')

#update w_events_max
def update_w_events(*arg):
    game_type = 'R' if w_gameType.value == 'regular' else 'P'
    game_ids_list = obj_extraction.get_game_ids(game_type, w_year.value + str(int(w_year.value)+1))
    
    game_data = obj_extraction.fetch_data(w_year.value, game_type, game_ids_list[w_game_id.value - 1][-4:])
    id_number = len(game_data['liveData']['plays']['allPlays'])
    w_events.max = id_number
w_gameType.observe(update_w_events, names='value')
w_year.observe(update_w_events, names='value')
w_game_id.observe(update_w_events, names='value')


out = widgets.interactive_output(f,{'w_year': w_year, 'w_gameType': w_gameType, 'w_game_id' : w_game_id, 'w_events' : w_events})
display(ui, out)




Output()